這章討論
* 何謂設計模式
* iterator 協定，最強設計模式之一
* 清單、集合、字典操作式
* 生產器與協程

# 設計模式簡介
---
設計模式嘗試將同樣的正規定義帶進軟體工程中的結構設計  
不同的設計模式解決不同的問題  
建構設計模式的人首先找出開發者在各種情況下會遇到的共同問題  
然後對於理想方案給出物件導向設計上的建議   

這一章討論 Iterator 模式  
此模式使得 Python 開發者提供多個語法來存取其物件導向原則  


## Iterator 
  
Iterator 是具有 `next()` 和 `done()` 方法的物件  
後者在序列中沒有其他項目時回傳 `True`  
Iterator 會向下方程式碼一樣逐個處理

```
while not iterator.done():
    item = iterator.next()
    # do something with the item
```
  
`__next__()` 迭代是一種特殊功能  
此方法可使用內建的 `next(iterator)` 存取  
`done` 方法則會迴圈完成時拋出 `StopIteration`

## 迭代協定
  
在 collections.abc 模組中的抽象基底 Iterator 類別定義 Python 中的迭代協定  
他必須要有 for 迴圈呼叫已從序列取得新元素的 `__next__()`  
每個迭代都需要滿足 Iterable 介面  
任何提供 `__iter__` 方法的類別都是可迭代的  

In [1]:
class CapitalIterable:
    def __init__(self, string):
        self.string = string
    def __iter__(self):
        return CapitalIterator(self.string)

class CapitalIterator:
    def __init__(self, string):
        self.words = [w.capitalize() for w in string.split()]
        self.index = 0
    def __next__(self):
        if self.index == len(self.words):
            raise StopIteration()
        word = self.words[self.index]
        self.index += 1
        return word
    def __iter__(self):
        return self

此程式的任務是逐個處理字串中的單字並將第一個字母轉成大寫後輸出

In [2]:
iterable = CapitalIterable('the quick brown fox jumps over the lazy dog')
iterator = iter(iterable)
while True:
    try:
        print(next(iterator))
    except StopIteration:
        break

The
Quick
Brown
Fox
Jumps
Over
The
Lazy
Dog


更簡單一點的語法

In [3]:
for i in iterable:
    print(i)

The
Quick
Brown
Fox
Jumps
Over
The
Lazy
Dog


# 操作式
---
操作式是簡單但很強的語法  
能夠讓我們以很少的程式轉換或過濾迭代物件  
運算結果可以是普通的清單、集合或字典

## 清單操作式
  
清單操作式 (Comprehension) 是 Python 中最有威力的工具之一  
讓我們檢視最常見的操作  
如下

In [4]:
input_strings = ['1','5','28' ,'131', '3']

output_integers = []
for num in input_strings:
    output_integers.append(int(num))

將上述程式改為操作式，如下範例

In [5]:
output_integers = [int(num) for num in input_strings]

程式縮減為一行  
且依舊保有可讀性  
效能也好
在操作大量項目時  
會比 for 迴圈快很多  
我們也能在操作式中加入 if 條件

In [6]:
output_ints = [int(n) for n in input_strings if len(n) < 3]

## 集合與字典操作式
  
操作式也能使用在集合與字典上  
下面是使用具名資料組建構的作者 / 書名 / 類型  
以及讀取所有撰寫特定類型書籍的作者的集合

In [7]:
from collections import namedtuple

Book = namedtuple("Book", "author title genre")
books = [
    Book("Pratchett", "Nightwatch", "fantasy"), 
    Book("Pratchett", "Thief of Time", "fantasy"), 
    Book("Le Guin", "The Dispossessed", "scifi"),  
    Book("Le Guin", "A Wizard ot Earthsea", "fantasy"),  
    Book("Turner", "The Thief", "fantasy"),   
    Book("Phillips", "Preston Diamond", "western"),  
    Book("Phillips", "Twice Upon A Time", "scifi"),  
]

fantasy_authors = {b.author for b in books if b.genre == 'fantasy'}
print(fantasy_authors)

{'Pratchett', 'Turner', 'Le Guin'}


上述程式利用大括弧去做操作式  
出來的結果會是集合(set)  
因此 Pratchett 不會被列出兩次

In [8]:
fantasy_title = {
    b.title: b for b in books if b.genre == 'fantasy'
}

fantasy_title

{'Nightwatch': Book(author='Pratchett', title='Nightwatch', genre='fantasy'),
 'Thief of Time': Book(author='Pratchett', title='Thief of Time', genre='fantasy'),
 'A Wizard ot Earthsea': Book(author='Le Guin', title='A Wizard ot Earthsea', genre='fantasy'),
 'The Thief': Book(author='Turner', title='The Thief', genre='fantasy')}

也可做成字典操作式

## 產生器運算式
  
有時我們想要處理新的序列而不置入系統記憶體新的清單、集合或字典  
只是想要處理一個項目且不需要建構最終容器物件  
就要靠產生器運算式  
跟操作式一樣的語法  
只是將操作式包裝在 `( )` 而不是 `[ ]` 或 `{ }` 
如下範例

```
import sys

inname = sys.args[1]
outname = sys.args[2]

with open(inname) as infile:
    with open(outname, 'w') as outfile:
        warnings = (l for l in infile if 'WARNING' in l)
        for l in warnings:
            outfile.write(l)
```
上方程式會將 log 中的 WARNING 的那幾行抓出來獨立寫在另外一個檔案  
對於處理資料量很大  
產生器運算式對記憶體與速度有很大的影響  
  
一般來說  
如果資料只需要過濾或轉換  
而不需要清單、集合或字典儲存時  
要盡可能的運用運算式產生器  
這樣會更有效率  

# 產生器
---
來看個例子  
```
import sys

inname, outname = sys.argv[1:3]

def warning_filter(insequence):
    for l in insequence:
        if 'WARNING' in l:
            yield l.replace('\tWARNING','')

with open(inname) as infile:
    with open(outname, 'w') as outfile:
        filter = warning_filter(infile)
        for l in filter:
            outfile.write(l)
```
什麼是 `yield` ??  
`yield` 是產生器的關鍵  
當 Python 看到函式中的 `yield` 時  
他會將函式包裝在物件中  
將 `yield` 陳述想成類似 `return`，離開函式時回傳一個值  
但當函式透過 `next()` 再次被呼叫時    
會從上次離開的位置繼續  
也就是 `yield` 的下一行  
而不是函式的最前面開始之處  
上面的程式因為 `yield` 後面沒有下一行了  
所以跳回 for 迴圈進行下一輪

In [9]:
def warning_filter(insequence):
    for l in insequence:
        if 'WARNING' in l:
            yield l.replace('\tWARNING','')

print(warning_filter([]))

<generator object warning_filter at 0x0000021D1FBC19C8>


可以看到此函式建構出一個特殊的產生器物件

In [78]:
def print_number(num_list):
    for item in num_list:
        yield item
        print(10)

num_list = [1,2,3,4,5]
a = print_number(num_list)

for i in a:
    print(i)

1
10
2
10
3
10
4
10
5
10


# 從其他iterable 產生項目
---
讓我們思考一個經典電腦問題：遍歷一個普通樹  
檔案系統是常見的樹資料結構

In [97]:
class File:
    def __init__(self, name) -> None:
        self.name = name

class Folder(File):
    def __init__(self, name) -> None:
        super().__init__(name)
        self.children = []

root = Folder('')
etc = Folder('etc')
root.children.append(etc)
etc.children.append(File('passwd'))
etc.children.append(File('groups'))
httpd = Folder('httpd')
etc.children.append(httpd)
httpd.children.append(File('http.conf'))
var = Folder('var')
root.children.append(var)
log = Folder('log')
var.children.append(log)
log.children.append(File('messages'))
log.children.append(File('kernel'))

def walk(file):
    if isinstance(file, Folder):
        yield file.name + '/'
        for f in file.children:
            yield from walk(f)
    else:
        yield file.name

result = walk(root)
print(next(result))
print(next(result))
print('------------------')
for r in result:
    print(r)

/
etc/
------------------
passwd
groups
httpd/
http.conf
var/
log/
messages
kernel


# 協程
---
協程 (coroutine) 是非常強的結構  
通常會和產生器搞混  
協程相當難理解  
且也不太常用    
有一些函式庫大量使用協程(主要用在同時性和非同步程式設計)  
下面是一個簡單的協程範例  

In [89]:
def tally():
    score = 0
    while True:
        increment = yield score
        score += increment

這個程式看起來無法運作  
我們先來檢視他的執行  
這個物件可用來球隊計分

In [91]:
white_sox = tally()
blue_jays = tally()

print("Game start!")
print('white_sox:', next(white_sox))
print('blue_jays:', next(blue_jays))
print("\nThe first inning")
print('white_sox:', white_sox.send(3))
print('blue_jays:', blue_jays.send(2))
print("\nThe second inning")
print('white_sox:', white_sox.send(2))
print('blue_jays:', blue_jays.send(4))

Game start!
white_sox: 0
blue_jays: 0

The first inning
white_sox: 3
blue_jays: 2

The second inning
white_sox: 5
blue_jays: 6


事情發生的順序是這樣的
1. `yield` 發生且產生器暫停
2. `send()`在函式之外觸發，並將 `send()` 內的值帶進函式裡，並且從剛剛停下的 `yield` 開始出發
3. `increment` 被賦予 `send()` 裡的數字
4. `score` 加上 `increment`
5. 繼續下一圈，碰到 `yield` 將值拋出，產生器暫停

現在我們有一個 Linux Log File 如下
> unrelated log messages  
sd 0:0:0:0 Attached Disk Drive  
unrelated log messages  
sd 0:0:0:0 (SERIAL=ZZ12345)  
unrelated log messages  
sd 0:0:0:0 [sda] Options  
unrelated log messages  
XFS ERROR [sda]  
unrelated log messages  
sd 2:0:0:1 Attached Disk Drive  
unrelated log messages  
sd 2:0:0:1 (SERIAL=ZZ67890)  
unrelated log messages  
sd 2:0:0:1 [sdb] Options  
unrelated log messages  
sd 3:0:1:8 Attached Disk Drive  
unrelated log messages  
sd 3:0:1:8 (SERIAL=WW11111)  
unrelated log messages  
sd 3:0:1:8 [sdc] Options  
unrelated log messages  
XFS ERROR [sdc]  
unrelated log messages  
  
我們要找出所有發生 XFS ERROR 的 SERIAL NUMBER  
程式如下

In [25]:
import re

def match_regex(filename, regex):
    with open(filename) as file:
        lines = file.readlines()
    for line in reversed(lines):
        match = re.match(regex, line)
        if match:
            regex = yield match.group(1)

def get_serial(filename):
    ERROR_RE = 'XFS ERROR (\[sd[a-z]\])'
    matcher = match_regex(filename, ERROR_RE)
    device = next(matcher)
    while True:
        bus = matcher.send(f'(sd \S+) {re.escape(device)}.*')
        serial = matcher.send(f'{bus} \(SERIAL=([^)]*)\)')
        yield serial
        device = matcher.send(ERROR_RE)
try:
    for serial_number in get_serial('EXAMPLE_LOG.log'):
        print(serial_number)
except RuntimeError:
    print('Done')

WW11111
ZZ12345
Done
